**Install required packages**

In [1]:
!pip install flask flask-ngrok nltk textblob scikit-learn pandas numpy transformers torch
!pip install pyngrok

In [2]:
# Download required NLTK data
import nltk
nltk.download('punkt')
nltk.download('vader_lexicon')
nltk.download('stopwords')

print("✅ All packages installed successfully!")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


✅ All packages installed successfully!


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


**Import libraries and setup environment**

In [3]:
import json
import random
import pickle
import numpy as np
import pandas as pd
from datetime import datetime
import re
from collections import defaultdict

In [4]:
# NLP Libraries
import nltk
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [5]:
# Flask for web interface
from flask import Flask, render_template, request, jsonify, redirect, url_for
from flask_ngrok import run_with_ngrok

# For Google Colab tunneling
from pyngrok import ngrok
import threading
import time

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


**Create training data**

In [6]:
# This creates the intents and responses for our chatbot
intents_data = {
    "intents": [
        {
            "tag": "greeting",
            "patterns": [
                "Hi", "Hello", "Hey", "Good morning", "Good evening", "What's up",
                "Greetings", "Howdy", "Hi there", "Hello there", "Good day"
            ],
            "responses": [
                "Hello! How can I help you today?",
                "Hi there! What can I do for you?",
                "Greetings! How may I assist you?",
                "Hello! I'm here to help. What do you need?",
                "Hey! What's on your mind?"
            ]
        },
        {
            "tag": "goodbye",
            "patterns": [
                "Bye", "Goodbye", "See you later", "Take care", "Farewell",
                "See you soon", "Catch you later", "Until next time", "Adios"
            ],
            "responses": [
                "Goodbye! Have a wonderful day!",
                "See you later! Take care!",
                "Farewell! It was nice chatting with you!",
                "Bye! Feel free to come back anytime!",
                "Take care! See you soon!"
            ]
        },
        {
            "tag": "thanks",
            "patterns": [
                "Thanks", "Thank you", "Appreciate it", "Thanks a lot",
                "Thank you so much", "Much appreciated", "Grateful"
            ],
            "responses": [
                "You're welcome!",
                "Happy to help!",
                "My pleasure!",
                "Glad I could assist you!",
                "Anytime! Feel free to ask if you need more help."
            ]
        },
        {
            "tag": "help",
            "patterns": [
                "Help", "I need help", "Can you help me", "Assist me",
                "I'm confused", "I don't understand", "What can you do"
            ],
            "responses": [
                "I'm here to help! You can ask me about various topics, and I'll do my best to assist you.",
                "Sure! I can help with questions, provide information, or just have a conversation. What do you need?",
                "Of course! I can assist with many things. What would you like help with?",
                "I'd be happy to help! What specific question or topic can I assist you with?"
            ]
        },
        {
            "tag": "weather",
            "patterns": [
                "What's the weather", "How's the weather", "Weather forecast",
                "Is it sunny", "Is it raining", "Temperature", "Climate"
            ],
            "responses": [
                "I don't have access to real-time weather data, but I'd recommend checking a weather app or website for accurate information!",
                "For current weather conditions, please check your local weather service or weather app.",
                "I can't provide real-time weather updates, but weather.com or your phone's weather app will have current conditions!"
            ]
        },
        {
            "tag": "personal",
            "patterns": [
                "How are you", "How do you feel", "Are you okay", "What's your name",
                "Who are you", "Tell me about yourself", "Your age"
            ],
            "responses": [
                "I'm doing well, thank you for asking! I'm an AI chatbot created to help and chat with you.",
                "I'm great! I'm here and ready to help you with whatever you need.",
                "I'm an AI assistant, so I don't have feelings in the traditional sense, but I'm functioning well and happy to chat!",
                "I'm your friendly AI chatbot! I'm here to help answer questions and have conversations."
            ]
        },
        {
            "tag": "joke",
            "patterns": [
                "Tell me a joke", "Make me laugh", "Say something funny",
                "Joke", "Humor", "Funny", "Entertain me"
            ],
            "responses": [
                "Why don't scientists trust atoms? Because they make up everything!",
                "I told my wife she was drawing her eyebrows too high. She seemed surprised!",
                "Why don't eggs tell jokes? They'd crack each other up!",
                "What do you call a fake noodle? An impasta!",
                "Why did the scarecrow win an award? He was outstanding in his field!"
            ]
        },
        {
            "tag": "compliment",
            "patterns": [
                "You're great", "You're awesome", "You're helpful", "Good job",
                "Well done", "You're smart", "I like you", "You're the best"
            ],
            "responses": [
                "Thank you so much! That means a lot to me!",
                "You're very kind! I'm glad I could help!",
                "Aww, thank you! You're pretty awesome yourself!",
                "I appreciate the compliment! It makes me happy to be helpful!",
                "That's so sweet of you to say! Thank you!"
            ]
        },
        {
            "tag": "time",
            "patterns": [
                "What time is it", "Current time", "Time", "What's the time",
                "Clock", "Hour", "Tell me the time"
            ],
            "responses": [
                "I don't have access to real-time clock information, but you can check the time on your device!",
                "Please check your device's clock for the current time.",
                "I can't tell you the exact time, but it should be displayed on your computer or phone!"
            ]
        }
    ]
}

In [7]:
# Save the intents data
with open('intents.json', 'w') as f:
    json.dump(intents_data, f, indent=2)

print("✅ Training data created and saved!")

✅ Training data created and saved!


**Sentiment Analyzer Class**

In [8]:
class SentimentAnalyzer:
    """
    This class analyzes the sentiment (emotion) of user messages
    It uses TextBlob to determine if a message is positive, negative, or neutral
    """

    def __init__(self):
        print("🎭 Sentiment Analyzer initialized!")

    def analyze_sentiment(self, text):
        """Analyze sentiment of the input text"""
        try:
            blob = TextBlob(text)
            polarity = blob.sentiment.polarity

            # Classify sentiment based on polarity score
            if polarity > 0.1:
                return 'positive'
            elif polarity < -0.1:
                return 'negative'
            else:
                return 'neutral'
        except:
            return 'neutral'

    def get_sentiment_emoji(self, sentiment):
        """Get emoji based on sentiment"""
        emoji_map = {
            'positive': '😊',
            'negative': '😔',
            'neutral': '😐'
        }
        return emoji_map.get(sentiment, '🤖')

    def adjust_response_for_sentiment(self, response, user_sentiment):
        """Adjust bot response based on user's sentiment"""
        if user_sentiment == 'negative':
            supportive_phrases = [
                "I understand that might be frustrating. ",
                "I'm sorry you're feeling that way. ",
                "Let me try to help you feel better. "
            ]
            return random.choice(supportive_phrases) + response
        elif user_sentiment == 'positive':
            positive_phrases = [
                "I'm glad you're feeling positive! ",
                "That's wonderful! ",
                "Great to hear! "
            ]
            return random.choice(positive_phrases) + response

        return response

**Intent Classifier Class**

In [9]:
class IntentClassifier:
    """
    This class identifies what the user wants (intent) from their message
    It uses machine learning to classify user inputs into predefined categories
    """

    def __init__(self):
        self.vectorizer = TfidfVectorizer(lowercase=True, stop_words='english')
        self.classifier = MultinomialNB()
        self.intents_data = None
        self.is_trained = False
        print("🧠 Intent Classifier initialized!")

    def load_intents(self, intents_file='intents.json'):
        """Load intents data from JSON file"""
        try:
            with open(intents_file, 'r') as f:
                self.intents_data = json.load(f)
            print(f"✅ Loaded {len(self.intents_data['intents'])} intent categories!")
            return True
        except Exception as e:
            print(f"❌ Error loading intents: {e}")
            return False

    def preprocess_text(self, text):
        """Clean and preprocess text"""
        # Convert to lowercase and remove special characters
        text = re.sub(r'[^a-zA-Z0-9\s]', '', text.lower())
        return text.strip()

    def train_model(self):
        """Train the intent classification model"""
        if not self.intents_data:
            print("❌ No intents data loaded!")
            return False

        # Prepare training data
        patterns = []
        labels = []

        for intent in self.intents_data['intents']:
            tag = intent['tag']
            for pattern in intent['patterns']:
                patterns.append(self.preprocess_text(pattern))
                labels.append(tag)

        print(f"📚 Training on {len(patterns)} examples...")

        # Vectorize the patterns
        X = self.vectorizer.fit_transform(patterns)

        # Train the classifier
        self.classifier.fit(X, labels)
        self.is_trained = True

        # Calculate and display accuracy
        predictions = self.classifier.predict(X)
        accuracy = accuracy_score(labels, predictions)
        print(f"✅ Model trained! Training accuracy: {accuracy:.2%}")

        return True

    def predict_intent(self, user_input):
        """Predict the intent of user input"""
        if not self.is_trained:
            return 'unknown', 0.0

        try:
            # Preprocess input
            processed_input = self.preprocess_text(user_input)

            # Vectorize input
            X = self.vectorizer.transform([processed_input])

            # Get prediction and probability
            prediction = self.classifier.predict(X)[0]
            probabilities = self.classifier.predict_proba(X)[0]
            confidence = max(probabilities)

            return prediction, confidence
        except Exception as e:
            print(f"❌ Error predicting intent: {e}")
            return 'unknown', 0.0

**Response Generator Class**

In [10]:
class ResponseGenerator:
    """
    This class generates appropriate responses based on the identified intent
    It selects responses from predefined templates and can modify them based on context
    """

    def __init__(self, intents_data):
        self.intents_data = intents_data
        self.response_history = []
        print("💬 Response Generator initialized!")

    def get_response(self, intent, confidence_threshold=0.3):
        """Generate response based on intent and confidence"""

        # If confidence is too low, return default response
        if confidence_threshold < 0.3:
            return self.get_fallback_response()

        # Find the intent in our data
        for intent_data in self.intents_data['intents']:
            if intent_data['tag'] == intent:
                responses = intent_data['responses']

                # Avoid repeating the same response
                available_responses = [r for r in responses if r not in self.response_history[-3:]]
                if not available_responses:
                    available_responses = responses

                selected_response = random.choice(available_responses)
                self.response_history.append(selected_response)

                # Keep history manageable
                if len(self.response_history) > 10:
                    self.response_history = self.response_history[-5:]

                return selected_response

        return self.get_fallback_response()

    def get_fallback_response(self):
        """Return a fallback response when intent is unclear"""
        fallback_responses = [
            "I'm not sure I understand. Could you please rephrase that?",
            "That's interesting! Could you tell me more?",
            "I'm still learning. Can you try asking in a different way?",
            "Hmm, I'm not quite sure what you mean. Could you clarify?",
            "I want to help! Could you be a bit more specific?"
        ]
        return random.choice(fallback_responses)

**Conversation Manager Class**

In [11]:
class ConversationManager:
    """
    This class manages the conversation flow and maintains context
    It keeps track of the conversation history and user preferences
    """

    def __init__(self):
        self.conversation_history = []
        self.user_preferences = {}
        self.session_start_time = datetime.now()
        print("📝 Conversation Manager initialized!")

    def add_interaction(self, user_input, bot_response, intent, sentiment, confidence):
        """Add a new interaction to the conversation history"""
        interaction = {
            'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            'user_input': user_input,
            'bot_response': bot_response,
            'intent': intent,
            'sentiment': sentiment,
            'confidence': confidence
        }

        self.conversation_history.append(interaction)

        # Keep only last 50 interactions to save memory
        if len(self.conversation_history) > 50:
            self.conversation_history = self.conversation_history[-25:]

    def get_conversation_stats(self):
        """Get statistics about the current conversation"""
        if not self.conversation_history:
            return {"total_messages": 0}

        total_messages = len(self.conversation_history)
        sentiments = [interaction['sentiment'] for interaction in self.conversation_history]
        most_common_sentiment = max(set(sentiments), key=sentiments.count) if sentiments else 'neutral'

        avg_confidence = sum(interaction['confidence'] for interaction in self.conversation_history) / total_messages

        return {
            'total_messages': total_messages,
            'most_common_sentiment': most_common_sentiment,
            'average_confidence': round(avg_confidence, 2),
            'session_duration': str(datetime.now() - self.session_start_time).split('.')[0]
        }

**Main ChatBot Class**

In [12]:
class DynamicAIChatbot:
    """
    Main chatbot class that coordinates all components
    This is the brain of our chatbot that brings everything together
    """

    def __init__(self):
        print("🤖 Initializing Dynamic AI Chatbot...")

        # Initialize all components
        self.intent_classifier = IntentClassifier()
        self.sentiment_analyzer = SentimentAnalyzer()
        self.conversation_manager = ConversationManager()
        self.response_generator = None

        # Chatbot status
        self.is_ready = False

        print("✅ Chatbot components initialized!")

    def initialize(self):
        """Initialize and train the chatbot"""
        print("🚀 Starting chatbot training process...")

        # Load intents
        if not self.intent_classifier.load_intents():
            return False

        # Train the model
        if not self.intent_classifier.train_model():
            return False

        # Initialize response generator
        self.response_generator = ResponseGenerator(self.intent_classifier.intents_data)

        self.is_ready = True
        print("🎉 Chatbot is ready to chat!")
        return True

    def get_response(self, user_input):
        """Main method to get chatbot response"""
        if not self.is_ready:
            return {
                'response': "Sorry, I'm still initializing. Please try again in a moment.",
                'intent': 'system',
                'sentiment': 'neutral',
                'confidence': 1.0
            }

        if not user_input or user_input.strip() == "":
            return {
                'response': "I didn't catch that. Could you please say something?",
                'intent': 'unknown',
                'sentiment': 'neutral',
                'confidence': 0.0
            }

        # Step 1: Analyze sentiment
        sentiment = self.sentiment_analyzer.analyze_sentiment(user_input)

        # Step 2: Classify intent
        intent, confidence = self.intent_classifier.predict_intent(user_input)

        # Step 3: Generate response
        base_response = self.response_generator.get_response(intent, confidence)

        # Step 4: Adjust response based on sentiment
        final_response = self.sentiment_analyzer.adjust_response_for_sentiment(
            base_response, sentiment
        )

        # Step 5: Log the interaction
        self.conversation_manager.add_interaction(
            user_input, final_response, intent, sentiment, confidence
        )

        return {
            'response': final_response,
            'intent': intent,
            'sentiment': sentiment,
            'confidence': round(confidence, 2),
            'emoji': self.sentiment_analyzer.get_sentiment_emoji(sentiment)
        }

    def get_stats(self):
        """Get chatbot statistics"""
        return self.conversation_manager.get_conversation_stats()

**Initialize the ChatBot**

In [13]:
print("🔄 Creating and initializing the chatbot...")
chatbot = DynamicAIChatbot()
success = chatbot.initialize()

if success:
    print("✅ Chatbot initialization successful!")
else:
    print("❌ Chatbot initialization failed!")

# Test the chatbot
print("\n🧪 Testing the chatbot...")
test_messages = ["Hello", "How are you?", "Tell me a joke", "Thanks", "Goodbye"]

for message in test_messages:
    response = chatbot.get_response(message)
    print(f"User: {message}")
    print(f"Bot: {response['response']}")
    print(f"Intent: {response['intent']}, Sentiment: {response['sentiment']}, Confidence: {response['confidence']}")
    print("-" * 50)

🔄 Creating and initializing the chatbot...
🤖 Initializing Dynamic AI Chatbot...
🧠 Intent Classifier initialized!
🎭 Sentiment Analyzer initialized!
📝 Conversation Manager initialized!
✅ Chatbot components initialized!
🚀 Starting chatbot training process...
✅ Loaded 9 intent categories!
📚 Training on 70 examples...
✅ Model trained! Training accuracy: 90.00%
💬 Response Generator initialized!
🎉 Chatbot is ready to chat!
✅ Chatbot initialization successful!

🧪 Testing the chatbot...
User: Hello
Bot: Hello! How can I help you today?
Intent: greeting, Sentiment: neutral, Confidence: 0.34
--------------------------------------------------
User: How are you?
Bot: I want to help! Could you be a bit more specific?
Intent: greeting, Sentiment: neutral, Confidence: 0.16
--------------------------------------------------
User: Tell me a joke
Bot: Hmm, I'm not quite sure what you mean. Could you clarify?
Intent: joke, Sentiment: neutral, Confidence: 0.23
----------------------------------------------

In [14]:
# Verify your chatbot is working
print("Chatbot ready status:", chatbot.is_ready)
if chatbot.is_ready:
    print("✅ Chatbot is ready for web integration!")

    # Quick test
    test_response = chatbot.get_response("Hello")
    print("Test response:", test_response['response'])
else:
    print("❌ Please run the chatbot initialization code first!")

Chatbot ready status: True
✅ Chatbot is ready for web integration!
Test response: Greetings! How may I assist you?


**Install Correct Dependencies**

In [15]:
# Let's clean up and install the correct packages
!pip uninstall flask-ngrok -y -q
!pip install flask==2.3.3 pyngrok==6.0.0 nest-asyncio -q

import nest_asyncio
nest_asyncio.apply()

print("✅ Clean installation completed!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.1/96.1 kB 4.3 MB/s eta 0:00:00
✅ Clean installation completed!


**Import Libraries**

In [16]:
from flask import Flask, render_template_string, request, jsonify
from pyngrok import ngrok
import json
import threading
import time
import requests
from IPython.display import display, HTML
import os

print("✅ Libraries imported successfully!")

✅ Libraries imported successfully!


**Verify your ChatBot**

In [17]:
# Make sure your chatbot is working properly
print("🔍 Checking chatbot status...")
print(f"Chatbot exists: {'chatbot' in locals()}")

if 'chatbot' in locals():
    print(f"Chatbot ready: {chatbot.is_ready}")

    if chatbot.is_ready:
        # Quick test
        test_response = chatbot.get_response("Hello")
        print(f"✅ Test response: {test_response['response']}")
        print("🎉 Chatbot is working perfectly!")
    else:
        print("❌ Chatbot not ready - please run initialization code first")
else:
    print("❌ Chatbot variable not found - please run your AI chatbot code first")

🔍 Checking chatbot status...
Chatbot exists: True
Chatbot ready: True
✅ Test response: Hello! I'm here to help. What do you need?
🎉 Chatbot is working perfectly!


**Fixed Flask Application Class**

In [18]:
class FixedChatbotFlaskApp:
    """
    Fixed Flask application wrapper with proper ngrok integration
    This version resolves the port argument error
    """

    def __init__(self, chatbot_instance):
        self.chatbot = chatbot_instance
        self.app = Flask(__name__)
        self.app.secret_key = 'chatbot-secret-key-2024-fixed'

        # Setup routes
        self.setup_routes()

        print("🌐 Fixed Flask application initialized!")

    def setup_routes(self):
        """Setup all the web routes for our application"""

        @self.app.route('/')
        def home():
            """Main chat interface"""
            return render_template_string(self.get_html_template())

        @self.app.route('/chat', methods=['POST'])
        def chat():
            """Handle chat messages from the frontend"""
            try:
                # Get user message from request
                data = request.get_json()
                user_message = data.get('message', '').strip()

                if not user_message:
                    return jsonify({
                        'response': 'Please enter a message.',
                        'intent': 'error',
                        'sentiment': 'neutral',
                        'confidence': 0.0,
                        'emoji': '⚠️'
                    })

                # Get response from chatbot
                response_data = self.chatbot.get_response(user_message)

                return jsonify(response_data)

            except Exception as e:
                print(f"❌ Error in chat route: {e}")
                return jsonify({
                    'response': 'Sorry, there was an error processing your message. Please try again.',
                    'intent': 'error',
                    'sentiment': 'neutral',
                    'confidence': 0.0,
                    'emoji': '❌'
                }), 500

        @self.app.route('/stats')
        def stats():
            """Get chatbot statistics"""
            try:
                stats_data = self.chatbot.get_stats()
                return jsonify(stats_data)
            except Exception as e:
                print(f"❌ Error getting stats: {e}")
                return jsonify({'error': 'Could not fetch statistics'}), 500

        @self.app.route('/health')
        def health():
            """Health check endpoint"""
            return jsonify({
                'status': 'healthy',
                'chatbot_ready': self.chatbot.is_ready,
                'timestamp': time.time()
            })

    def get_html_template(self):
        """Returns the complete HTML template for the chat interface"""
        return '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>🤖 AI Chatbot - Live</title>
    <link href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.0.0/css/all.min.css" rel="stylesheet">
    <style>
        * {
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }

        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            min-height: 100vh;
            display: flex;
            justify-content: center;
            align-items: center;
            padding: 10px;
        }

        .chat-container {
            width: 100%;
            max-width: 450px;
            height: 85vh;
            max-height: 700px;
            background: white;
            border-radius: 20px;
            box-shadow: 0 25px 50px rgba(0,0,0,0.25);
            display: flex;
            flex-direction: column;
            overflow: hidden;
            animation: slideUp 0.6s ease-out;
        }

        @keyframes slideUp {
            from { opacity: 0; transform: translateY(30px); }
            to { opacity: 1; transform: translateY(0); }
        }

        .chat-header {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 20px;
            text-align: center;
            position: relative;
        }

        .chat-header h1 {
            font-size: 24px;
            margin-bottom: 8px;
            font-weight: 600;
        }

        .chat-header .subtitle {
            font-size: 14px;
            opacity: 0.9;
            display: flex;
            align-items: center;
            justify-content: center;
            gap: 8px;
        }

        .status-indicator {
            width: 12px;
            height: 12px;
            background: #4ade80;
            border-radius: 50%;
            animation: pulse 2s infinite;
        }

        @keyframes pulse {
            0%, 100% { opacity: 1; transform: scale(1); }
            50% { opacity: 0.7; transform: scale(1.1); }
        }

        .chat-messages {
            flex: 1;
            padding: 20px;
            overflow-y: auto;
            background: linear-gradient(to bottom, #f8f9fa, #e9ecef);
            scroll-behavior: smooth;
        }

        .welcome-message {
            text-align: center;
            padding: 30px 20px;
            color: #6b7280;
            background: white;
            border-radius: 15px;
            box-shadow: 0 4px 15px rgba(0,0,0,0.1);
            margin-bottom: 20px;
        }

        .welcome-message h3 {
            color: #374151;
            margin-bottom: 10px;
            font-size: 20px;
        }

        .welcome-message p {
            font-size: 14px;
            line-height: 1.6;
            color: #6b7280;
        }

        .message {
            margin-bottom: 20px;
            animation: messageSlide 0.5s ease-out;
        }

        @keyframes messageSlide {
            from { opacity: 0; transform: translateY(15px); }
            to { opacity: 1; transform: translateY(0); }
        }

        .user-message {
            text-align: right;
        }

        .bot-message {
            text-align: left;
        }

        .message-bubble {
            display: inline-block;
            padding: 14px 20px;
            border-radius: 20px;
            max-width: 80%;
            word-wrap: break-word;
            position: relative;
            font-size: 14px;
            line-height: 1.5;
        }

        .user-message .message-bubble {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            box-shadow: 0 4px 15px rgba(102, 126, 234, 0.3);
        }

        .bot-message .message-bubble {
            background: white;
            color: #333;
            border: 2px solid #e2e8f0;
            box-shadow: 0 2px 10px rgba(0,0,0,0.1);
        }

        .message-info {
            font-size: 11px;
            opacity: 0.7;
            margin-top: 8px;
            display: flex;
            align-items: center;
            gap: 8px;
        }

        .user-message .message-info {
            justify-content: flex-end;
        }

        .confidence-badge {
            background: #f1f5f9;
            padding: 2px 8px;
            border-radius: 10px;
            font-size: 10px;
            font-weight: 600;
        }

        .confidence-high { background: #dcfce7; color: #16a34a; }
        .confidence-medium { background: #fef3c7; color: #d97706; }
        .confidence-low { background: #fee2e2; color: #dc2626; }

        .chat-input {
            padding: 20px;
            background: white;
            border-top: 1px solid #e2e8f0;
            display: flex;
            gap: 12px;
            align-items: center;
        }

        .message-input {
            flex: 1;
            padding: 14px 20px;
            border: 2px solid #e2e8f0;
            border-radius: 25px;
            outline: none;
            font-size: 14px;
            transition: all 0.3s ease;
        }

        .message-input:focus {
            border-color: #667eea;
            box-shadow: 0 0 0 3px rgba(102, 126, 234, 0.1);
        }

        .send-button {
            padding: 14px 22px;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            border: none;
            border-radius: 25px;
            cursor: pointer;
            font-size: 14px;
            font-weight: 600;
            transition: all 0.3s ease;
            display: flex;
            align-items: center;
            gap: 8px;
            min-width: 90px;
            justify-content: center;
        }

        .send-button:hover:not(:disabled) {
            transform: translateY(-2px);
            box-shadow: 0 8px 25px rgba(102, 126, 234, 0.4);
        }

        .send-button:disabled {
            opacity: 0.6;
            cursor: not-allowed;
            transform: none;
        }

        .typing-indicator {
            display: none;
            padding: 12px 20px;
            background: white;
            border: 2px solid #e2e8f0;
            border-radius: 20px;
            margin-bottom: 20px;
            width: fit-content;
            box-shadow: 0 2px 10px rgba(0,0,0,0.1);
        }

        .typing-dots {
            display: flex;
            gap: 4px;
            align-items: center;
        }

        .typing-dots span {
            width: 8px;
            height: 8px;
            background: #667eea;
            border-radius: 50%;
            animation: typingDots 1.4s infinite ease-in-out;
        }

        .typing-dots span:nth-child(2) { animation-delay: 0.2s; }
        .typing-dots span:nth-child(3) { animation-delay: 0.4s; }

        @keyframes typingDots {
            0%, 80%, 100% { transform: scale(0.8); opacity: 0.5; }
            40% { transform: scale(1.2); opacity: 1; }
        }

        .connection-status {
            position: fixed;
            top: 20px;
            right: 20px;
            padding: 10px 15px;
            background: #10b981;
            color: white;
            border-radius: 20px;
            font-size: 12px;
            font-weight: 600;
            box-shadow: 0 4px 15px rgba(0,0,0,0.2);
            z-index: 1000;
        }

        /* Mobile Responsive */
        @media (max-width: 500px) {
            body { padding: 5px; }
            .chat-container {
                height: 95vh;
                border-radius: 15px;
            }
            .message-bubble { max-width: 85%; }
            .connection-status {
                top: 10px;
                right: 10px;
                font-size: 11px;
                padding: 8px 12px;
            }
        }

        /* Scrollbar */
        .chat-messages::-webkit-scrollbar {
            width: 6px;
        }

        .chat-messages::-webkit-scrollbar-track {
            background: transparent;
        }

        .chat-messages::-webkit-scrollbar-thumb {
            background: #cbd5e1;
            border-radius: 3px;
        }

        .loading-spinner {
            display: inline-block;
            width: 12px;
            height: 12px;
            border: 2px solid #f3f3f3;
            border-top: 2px solid #667eea;
            border-radius: 50%;
            animation: spin 1s linear infinite;
        }

        @keyframes spin {
            0% { transform: rotate(0deg); }
            100% { transform: rotate(360deg); }
        }
    </style>
</head>
<body>
    <div class="connection-status">
        <i class="fas fa-wifi"></i> Connected
    </div>

    <div class="chat-container">
        <div class="chat-header">
            <h1><i class="fas fa-robot"></i> AI Assistant</h1>
            <div class="subtitle">
                <span class="status-indicator"></span>
                Online & Ready to Chat
            </div>
        </div>

        <div class="chat-messages" id="chatMessages">
            <div class="welcome-message">
                <h3>👋 Hello! I'm your AI Assistant</h3>
                <p>I can help answer questions, have conversations, tell jokes, and much more. What would you like to talk about today?</p>
            </div>
        </div>

        <div class="typing-indicator" id="typingIndicator">
            <div class="typing-dots">
                <span></span>
                <span></span>
                <span></span>
            </div>
        </div>

        <div class="chat-input">
            <input
                type="text"
                id="messageInput"
                class="message-input"
                placeholder="Type your message here..."
                autocomplete="off"
                maxlength="500"
            >
            <button onclick="sendMessage()" id="sendButton" class="send-button">
                <i class="fas fa-paper-plane"></i>
                Send
            </button>
        </div>
    </div>

    <script>
        // Enhanced JavaScript for better user experience
        const chatMessages = document.getElementById('chatMessages');
        const messageInput = document.getElementById('messageInput');
        const sendButton = document.getElementById('sendButton');
        const typingIndicator = document.getElementById('typingIndicator');

        // Initialize
        document.addEventListener('DOMContentLoaded', function() {
            messageInput.focus();
            console.log('🤖 AI Chatbot loaded successfully!');
        });

        // Add message to chat
        function addMessage(message, isUser = false, extraInfo = null) {
            const welcomeMsg = document.querySelector('.welcome-message');
            if (welcomeMsg) {
                welcomeMsg.style.animation = 'fadeOut 0.3s ease-out';
                setTimeout(() => welcomeMsg.remove(), 300);
            }

            const messageDiv = document.createElement('div');
            messageDiv.className = `message ${isUser ? 'user-message' : 'bot-message'}`;

            const now = new Date();
            const timeString = now.toLocaleTimeString([], {hour: '2-digit', minute:'2-digit'});

            let infoHTML = '';
            if (!isUser && extraInfo) {
                const confidence = Math.round(extraInfo.confidence * 100);
                const confidenceClass = confidence > 70 ? 'confidence-high' :
                                      confidence > 40 ? 'confidence-medium' : 'confidence-low';

                infoHTML = `
                    <div class="message-info">
                        ${extraInfo.emoji} ${extraInfo.sentiment} • ${timeString}
                        <span class="confidence-badge ${confidenceClass}">${confidence}% sure</span>
                    </div>
                `;
            } else {
                infoHTML = `<div class="message-info">${isUser ? 'You' : 'AI'} • ${timeString}</div>`;
            }

            messageDiv.innerHTML = `
                <div class="message-bubble">${message}</div>
                ${infoHTML}
            `;

            chatMessages.appendChild(messageDiv);
            scrollToBottom();
        }

        function scrollToBottom() {
            chatMessages.scrollTop = chatMessages.scrollHeight;
        }

        function showTyping() {
            typingIndicator.style.display = 'block';
            scrollToBottom();
        }

        function hideTyping() {
            typingIndicator.style.display = 'none';
        }

        // Enhanced send message with better error handling
        async function sendMessage() {
            const message = messageInput.value.trim();
            if (!message || sendButton.disabled) return;

            addMessage(message, true);

            messageInput.value = '';
            sendButton.disabled = true;
            sendButton.innerHTML = '<div class="loading-spinner"></div> Thinking...';

            showTyping();

            try {
                const response = await fetch('/chat', {
                    method: 'POST',
                    headers: {
                        'Content-Type': 'application/json',
                    },
                    body: JSON.stringify({ message: message }),
                    timeout: 10000  // 10 second timeout
                });

                if (!response.ok) {
                    throw new Error(`Server error: ${response.status}`);
                }

                const data = await response.json();

                hideTyping();

                // Simulate natural typing delay
                setTimeout(() => {
                    addMessage(data.response, false, {
                        emoji: data.emoji,
                        sentiment: data.sentiment,
                        confidence: data.confidence
                    });
                }, 500);

            } catch (error) {
                hideTyping();
                console.error('Chat error:', error);

                let errorMessage = "I'm having trouble connecting right now. Please try again! 🔄";
                if (error.message.includes('timeout')) {
                    errorMessage = "That took too long to process. Could you try rephrasing your message? ⏱️";
                }

                addMessage(errorMessage, false, {
                    emoji: '❌',
                    sentiment: 'neutral',
                    confidence: 0.5
                });
            }

            sendButton.disabled = false;
            sendButton.innerHTML = '<i class="fas fa-paper-plane"></i> Send';
            messageInput.focus();
        }

        // Better keyboard handling
        messageInput.addEventListener('keypress', function(e) {
            if (e.key === 'Enter' && !e.shiftKey) {
                e.preventDefault();
                sendMessage();
            }
        });

        // Auto-focus management
        document.addEventListener('click', function(e) {
            if (e.target.tagName !== 'INPUT' && e.target.tagName !== 'BUTTON') {
                messageInput.focus();
            }
        });

        // Connection status monitoring
        async function checkConnection() {
            try {
                const response = await fetch('/health');
                const statusEl = document.querySelector('.connection-status');

                if (response.ok) {
                    statusEl.innerHTML = '<i class="fas fa-wifi"></i> Connected';
                    statusEl.style.background = '#10b981';
                } else {
                    throw new Error('Health check failed');
                }
            } catch (error) {
                const statusEl = document.querySelector('.connection-status');
                statusEl.innerHTML = '<i class="fas fa-wifi"></i> Reconnecting...';
                statusEl.style.background = '#f59e0b';
            }
        }

        // Check connection every 30 seconds
        setInterval(checkConnection, 30000);

        console.log('🚀 Chat interface fully loaded and ready!');
    </script>
</body>
</html>
        '''

    def start_with_ngrok(self, port=5000):
        """Start Flask app with ngrok tunnel - FIXED VERSION"""

        print(f"🚀 Starting Flask application on port {port}...")

        try:
            # Kill any existing ngrok tunnels
            ngrok.kill()

            # Start ngrok tunnel
            print("🌐 Creating ngrok tunnel...")
            public_url = ngrok.connect(port)

            print(f"✅ Ngrok tunnel created!")
            print(f"🌍 PUBLIC URL: {public_url}")
            print(f"📱 Your chatbot is now live and accessible from anywhere!")
            print(f"🔗 Click here: {public_url}")

            # Display clickable link in Colab
            display(HTML(f'''
                <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                           padding: 20px; border-radius: 10px; margin: 10px 0; text-align: center;">
                    <h2 style="color: white; margin-bottom: 10px;">🎉 Your AI Chatbot is Live!</h2>
                    <a href="{public_url}" target="_blank"
                       style="background: white; color: #667eea; padding: 15px 30px;
                              border-radius: 25px; text-decoration: none; font-weight: bold;
                              font-size: 16px; display: inline-block;">
                        🚀 OPEN YOUR CHATBOT
                    </a>
                    <p style="color: white; margin-top: 15px; font-size: 14px;">
                        Share this URL with anyone to let them chat with your AI!
                    </p>
                </div>
            '''))

            # Start Flask app (this will run indefinitely)
            print("⚡ Starting Flask server...")
            self.app.run(host='0.0.0.0', port=port, debug=False, use_reloader=False)

        except Exception as e:
            print(f"❌ Error starting application: {e}")
            print("💡 Trying alternative method...")

            # Alternative: start without ngrok first, then create tunnel
            try:
                # Start Flask in a thread
                flask_thread = threading.Thread(
                    target=lambda: self.app.run(host='0.0.0.0', port=port, debug=False)
                )
                flask_thread.daemon = True
                flask_thread.start()

                # Wait a moment for Flask to start
                time.sleep(3)

                # Create ngrok tunnel
                public_url = ngrok.connect(port)
                print(f"✅ Alternative method successful!")
                print(f"🌍 PUBLIC URL: {public_url}")

                display(HTML(f'''
                    <div style="background: #10b981; color: white; padding: 15px;
                               border-radius: 8px; margin: 10px 0; text-align: center;">
                        <h3>✅ Chatbot is running!</h3>
                        <a href="{public_url}" target="_blank"
                           style="color: white; font-weight: bold; font-size: 16px;">
                            Click here: {public_url}
                        </a>
                    </div>
                '''))

                # Keep the main thread alive
                while True:
                    time.sleep(60)
                    print(f"🟢 Chatbot still running at: {public_url}")

            except Exception as e2:
                print(f"❌ Alternative method also failed: {e2}")
                print("💡 Please try restarting the kernel and running again.")

**Fixed Integration Functions**

In [19]:
def create_web_interface_fixed(chatbot_instance):
    """
    FIXED VERSION: Create and return a Flask web interface for your chatbot
    """

    print("🔗 Creating FIXED web interface for your chatbot...")

    # Verify chatbot is ready
    if not chatbot_instance or not hasattr(chatbot_instance, 'is_ready') or not chatbot_instance.is_ready:
        print("❌ Error: Chatbot is not initialized or ready!")
        print("💡 Make sure you run the chatbot initialization code first.")
        print("💡 Your chatbot variable should have 'is_ready' attribute set to True.")
        return None

    # Create Flask app
    flask_app = FixedChatbotFlaskApp(chatbot_instance)

    print("✅ FIXED web interface created successfully!")
    print("📝 Next step: Run web_app.start_with_ngrok() to get your public URL")

    return flask_app

def quick_start_web_chatbot_fixed(chatbot_instance, port=5000):
    """
    FIXED VERSION: Quick start function - creates and runs web interface in one step
    """

    print("🚀 FIXED Quick Start for Web Chatbot...")

    web_app = create_web_interface_fixed(chatbot_instance)

    if web_app:
        print("🌟 Starting web server with fixed ngrok integration...")
        web_app.start_with_ngrok(port=port)
    else:
        print("❌ Failed to create web interface. Check your chatbot initialization.")

**Testing and Verification**

In [20]:
def test_chatbot_connection(chatbot_instance):
    """Test if chatbot is properly initialized"""
    print("🧪 Testing chatbot connection...")

    if not chatbot_instance:
        print("❌ Chatbot instance is None")
        return False

    if not hasattr(chatbot_instance, 'is_ready'):
        print("❌ Chatbot missing 'is_ready' attribute")
        return False

    if not chatbot_instance.is_ready:
        print("❌ Chatbot is not ready (is_ready = False)")
        return False

    try:
        test_response = chatbot_instance.get_response("Hello test")
        if test_response and 'response' in test_response:
            print("✅ Chatbot connection test passed!")
            print(f"✅ Test response: {test_response['response'][:50]}...")
            return True
        else:
            print("❌ Chatbot returned invalid response format")
            return False

    except Exception as e:
        print(f"❌ Chatbot test failed with error: {e}")
        return False

print("""
🎯 FIXED FLASK APPLICATION READY!

TO START YOUR WEB CHATBOT (FIXED VERSION):

1. FIRST - Test your chatbot:
   ```python
   test_chatbot_connection(chatbot)
   ```

2. THEN - Start the web interface:
   ```python
   web_app = create_web_interface_fixed(chatbot)
   web_app.start_with_ngrok()
   ```

3. OR - Quick start in one command:
   ```python
   quick_start_web_chatbot_fixed(chatbot)
   ```

🔧 FIXES APPLIED:
✅ Removed problematic flask-ngrok dependency
✅ Fixed port parameter issue
✅ Added proper error handling
✅ Improved ngrok integration
✅ Better connection testing
✅ Enhanced user interface

🌐 YOUR CHATBOT WILL BE AVAILABLE AT:
- A public ngrok URL (works from anywhere!)
- Clickable link will appear after starting
- Mobile-friendly responsive design
""")


🎯 FIXED FLASK APPLICATION READY!

TO START YOUR WEB CHATBOT (FIXED VERSION):

1. FIRST - Test your chatbot:
   ```python
   test_chatbot_connection(chatbot)
   ```

2. THEN - Start the web interface:
   ```python
   web_app = create_web_interface_fixed(chatbot)
   web_app.start_with_ngrok()
   ```

3. OR - Quick start in one command:
   ```python
   quick_start_web_chatbot_fixed(chatbot)
   ```

🔧 FIXES APPLIED:
✅ Removed problematic flask-ngrok dependency
✅ Fixed port parameter issue
✅ Added proper error handling
✅ Improved ngrok integration
✅ Better connection testing
✅ Enhanced user interface

🌐 YOUR CHATBOT WILL BE AVAILABLE AT:
- A public ngrok URL (works from anywhere!)
- Clickable link will appear after starting
- Mobile-friendly responsive design



**Start your Web ChatBot**

In [21]:
# Install Gradio (free and no authentication needed)
!pip install gradio -q

import gradio as gr
import time

def chat_with_ai(message, history):
    """Gradio chat interface"""
    if not message.strip():
        return history, ""

    # Get bot response
    try:
        response_data = chatbot.get_response(message)

        # Format response with emoji and confidence
        bot_response = f"{response_data['response']} {response_data['emoji']}"
        confidence_info = f"({response_data['sentiment']} • {int(response_data['confidence']*100)}% confident)"

        # Add to history
        history.append([message, f"{bot_response}\n\n*{confidence_info}*"])

        return history, ""

    except Exception as e:
        history.append([message, f"Sorry, I encountered an error: {str(e)}"])
        return history, ""

# Create beautiful Gradio interface
with gr.Blocks(
    title="🤖 Dynamic AI Chatbot",
    theme=gr.themes.Soft(),
    css="""
    .gradio-container {
        max-width: 700px !important;
        margin: auto !important;
    }
    """
) as demo:

    gr.Markdown("""
    # 🤖 Dynamic AI Chatbot
    ### Your Personal AI Assistant - Powered by Advanced NLP

    Ask me anything! I can help with questions, have conversations, tell jokes, and more.
    """)

    chatbot_ui = gr.Chatbot(
        height=500,
        show_label=False,
        avatar_images=["https://api.dicebear.com/7.x/personas/svg?seed=user",
                      "https://api.dicebear.com/7.x/bottts/svg?seed=bot"]
    )

    with gr.Row():
        msg = gr.Textbox(
            placeholder="Type your message here...",
            show_label=False,
            scale=4
        )
        send_btn = gr.Button("Send 🚀", scale=1, variant="primary")

    clear_btn = gr.Button("Clear Chat 🗑️", variant="secondary")

    # Event handlers
    msg.submit(chat_with_ai, [msg, chatbot_ui], [chatbot_ui, msg])
    send_btn.click(chat_with_ai, [msg, chatbot_ui], [chatbot_ui, msg])
    clear_btn.click(lambda: ([], ""), None, [chatbot_ui, msg], queue=False)

# Launch with public link (FREE!)
print("🚀 Launching your AI Chatbot...")
public_url = demo.launch(
    share=True,  # Creates free public URL
    server_name="0.0.0.0",
    server_port=7860,
    show_api=False,
    quiet=False
)

print(f"✅ Your chatbot is now LIVE!")
print(f"🌍 Public URL will appear above this message!")
print(f"📱 Share the gradio link with anyone to let them chat with your AI!")

/tmp/ipython-input-2671707775.py:48: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot_ui = gr.Chatbot(


🚀 Launching your AI Chatbot...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://88f7ff45b2f8a6d1a1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


✅ Your chatbot is now LIVE!
🌍 Public URL will appear above this message!
📱 Share the gradio link with anyone to let them chat with your AI!
